In [1]:
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
from datetime import timedelta as delta
from glob import glob
import numpy as np
import xarray as xr
import os

In [2]:
ddir = "/data/oceanparcels/input_data/NEMO-MEDUSA/ORCA0083-N006/"
#odir = "../../results/output_data"
#fname = os.path.join(odir,"galapagosparticles_testrun.nc")
fname = "galapagosparticles_testrun.nc"

# set field, only year 2000
ufiles = sorted(glob(ddir+'means/ORCA0083-N06_200[0-0]*d05U.nc'))
vfiles = [u.replace('05U.nc', '05V.nc') for u in ufiles]
meshfile = glob(ddir+'domain/coordinates.nc')
nemofiles = {'U': {'lon': meshfile, 'lat': meshfile, 'data': ufiles},
             'V': {'lon': meshfile, 'lat': meshfile, 'data': vfiles}}
nemovariables = {'U': 'uo', 'V': 'vo'}
nemodimensions = {'lon': 'glamf', 'lat': 'gphif', 'time': 'time_counter'}
fieldset = FieldSet.from_nemo(nemofiles, nemovariables, nemodimensions)

fU = fieldset.U
fieldset.computeTimeChunk(fU.grid.time[0], 1)    #forward
#fieldset.computeTimeChunk(fU.grid.time[-1], -1)  #backward

         It will be opened with no decoding. Filling values might be wrongly parsed.
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauuo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauuo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 

/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauuo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauuo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6

/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauuo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauuo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6

/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauuo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauuo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6

/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauvo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauvo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6

/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauvo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauvo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauvo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python

/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauvo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauvo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6

/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauvo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauvo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,


864000.0

In [3]:
#initialize where to start particles
galapagos_extent = [-91.8, -89, -1.4, 0.7]
startlon, startlat = np.meshgrid(np.arange(galapagos_extent[0], galapagos_extent[1], 0.2),
                                 np.arange(galapagos_extent[2], galapagos_extent[3], 0.2))

In [4]:
#functions to add to the kernel
def Age(fieldset, particle, time):
    particle.age = particle.age + math.fabs(particle.dt)
    if particle.age > 30*86400:
        particle.delete()

def WrapParticle(particle, fieldset, time):
    if particle.lon < -530:
        particle.lon += 360        

#additional features of the particles        
class GalapagosParticle(JITParticle):
    age = Variable('age', initial = 0.)            

In [5]:
# set particle conditions
pset = ParticleSet(fieldset=fieldset,
                   pclass=GalapagosParticle,
                   lon=startlon,
                   lat=startlat,
                   time=fU.grid.time[0])

In [7]:
outfile = pset.ParticleFile(name=fname, outputdt=delta(days=1))

pset.execute(AdvectionRK4+pset.Kernel(Age)+WrapParticle,
             dt=delta(hours=1),
             output_file=outfile)

outfile.close()

INFO: Compiled GalapagosParticleAdvectionRK4AgeWrapParticle ==> /tmp/parcels-302994/bba2d301713fdf5e905b657fae614f59_0.so
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'uo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauuo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'vo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'tauvo' has multiple fill values {0.0, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,
/usr/local/lib/python3.6/site-packages/xarray/conventions.py:494: SerializationWarning: variable

TypeError: __init__() got an unexpected keyword argument 'max_value'